# Data Preparation 

### Linking Anthropology's Data and Archives (LADA)

### AI-Generated Linked Data Evaluation (part I)

In [1]:
import config
import utils
import pandas as pd
import numpy as np
from pathlib import Path
import os
import re

Read in CSV data as a pandas DataFrame, where `f` is the name of the data file:

In [28]:
# Uncomment one of the following code lines or replace with your own
# path to the folder (directory) where the data file f is located
# ------------------------------------
# data_dir = config.task1_data
data_dir = config.playgrd1_data
# data_dir = config.playgrd3_data
# data_dir = "path/to/your/data/"

# The name of the data file
# ------------------------------------
# f = "4-HDataExperimentAssignmentsAndOutcomes_Outcomes_Task1.csv"
f = "4-HDataExperimentAssignmentsAndOutcomes_Playground_Task1.csv"
# f = "4-HDataExperimentAssignmentsAndOutcomes_Playground_Task3.csv"
# f = "your_data_file_name.csv"

In [29]:
df = pd.read_csv(data_dir + f)
df.head(10)

,ID,Filename,"Transkribus Transcription (or link to separate doc, if too long)","GPT Transcription (or link to separate doc, if too long)",GPT Prompt,most accurate transcript?,Metadata record,Schema.org Record,CIDOC-CRM Record,prompts to GPT,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,NaN,"Ann Aldrich, Mushmilling Hundred, Kent County,...",part County Deliway State\nIn quesereson taken...,Kent County Delivery State\nInquisition taken ...,"""transcribe this""",ChatGPT,<dublincore>\n <dc:title>Inquisition Report...,"{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",“create a Dublin Core metadata record”\n“creat...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"Edward Allen, No Hundred Listed, Kent County, ...",Personly Append before Jame Richarson\nCa...,Kent County\n\nPersonally appeared before Jame...,"""transcribe this""",ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",“create a Dublin Core metadata record in xml”\...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"Mirandy Messick, Nanticoke Hundred, Sussex Cou...",r Mutlean Indented & taken in Nantcke\nty\nSup...,Inquest convened\nState of Alabama\n\nA pre-mi...,"""transcribe this""",ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": ""https://www.cidoc-crm.org/ci...",“create a Dublin Core metadata record in XML b...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"James Abbott, Mispillion Hundred, Kent County,...",Renl County by Anedmstion indented and take\nS...,"\nKent County, State of Delaware\n\nAn inquisi...","""transcribe this""",ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<rdf:R...","{\n ""@context"": ""http://schema.org"",\n ""@typ...","{\n ""@context"": {\n ""@vocab"": ""http://www....",“create a Dublin core record in xml based on b...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,"Merritt Allee, Little Creek Hundred, Kent Coun...",Jient Crupty S.\nAn Inquiſte Aaten Le Ealeenth...,Front County S.S.\nInquisition taken at Greenw...,"""transcribe this""",ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": ""https://www.cidoc-crm.org/ci...",“create a Dublin Core record in XML”\n“create ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,wrhsms06185.pdf,3\nat\n\n55\n\nEaa\no 30\n55\n285\nStotement 9...,Transcription of Diana Stedman's Account of Ja...,Create a transcription for these images.,ChatGPT,"<metadata xmlns:dc=""http://purl.org/dc/element...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",Prompt 1: Can you create a Dublin Core XML met...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NASM-NASM.1993.0060-M0000028-00110.jpg,NaN,Image Transcription / Description\nTitle: Afri...,Create a transcription for this image. This im...,ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","""{\n """"@context"""": """"https://schema.org"""",\n ...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",Prompt 1: Can you create an xml metadata recor...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,"William Beynon to Franz Boas. 1933-05-22, May ...",Task 3 - Transkribus - Ugoma text_155693_OBJ.jpeg,ID_3_William Beynon Letter to Franz Boas 1933_...,Create a transcription for this text. This tex...,ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",Prompt 1: Create a transcription of this lette...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,p262401coll004_252_full.jpg,NaN,Image Description / Transcription:\n\nThe imag...,Create

Remove empty columns from the DataFrame:

In [30]:
print(df.shape)
df = df.dropna(axis=1, how='all')
print(df.shape)
df.head()

(999, 28)
(999, 9)


,Filename,"Transkribus Transcription (or link to separate doc, if too long)","GPT Transcription (or link to separate doc, if too long)",GPT Prompt,most accurate transcript?,Metadata record,Schema.org Record,CIDOC-CRM Record,prompts to GPT
0,"Ann Aldrich, Mushmilling Hundred, Kent County,...",part County Deliway State\nIn quesereson taken...,Kent County Delivery State\nInquisition taken ...,"""transcribe this""",ChatGPT,<dublincore>\n <dc:title>Inquisition Report...,"{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",“create a Dublin Core metadata record”\n“creat...
1,"Edward Allen, No Hundred Listed, Kent County, ...",Personly Append before Jame Richarson\nCa...,Kent County\n\nPersonally appeared before Jame...,"""transcribe this""",ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",“create a Dublin Core metadata record in xml”\...
2,"Mirandy Messick, Nanticoke Hundred, Sussex Cou...",r Mutlean Indented & taken in Nantcke\nty\nSup...,Inquest convened\nState of Alabama\n\nA pre-mi...,"""transcribe this""",ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": ""https://www.cidoc-crm.org/ci...",“create a Dublin Core metadata record in XML b...
3,"James Abbott, Mispillion Hundred, Kent County,...",Renl County by Anedmstion indented and take\nS...,"\nKent County, State of Delaware\n\nAn inquisi...","""transcribe this""",ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<rdf:R...","{\n ""@context"": ""http://schema.org"",\n ""@typ...","{\n ""@context"": {\n ""@vocab"": ""http://www....",“create a Dublin core record in xml based on b...
4,"Merritt Allee, Little Creek Hundred, Kent Coun...",Jient Crupty S.\nAn Inquiſte Aaten Le Ealeenth...,Front County S.S.\nInquisition taken at Greenw...,"""transcribe this""",ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": ""https://www.cidoc-crm.org/ci...",“create a Dublin Core record in XML”\n“create ...


Remove rows without a metadata record from the DataFrame:

In [31]:
# df.dropna(how="all", inplace=True)
df.dropna(subset=["Metadata record"], inplace=True)
print(df.shape)
df.tail()

(27, 9)


,Filename,"Transkribus Transcription (or link to separate doc, if too long)","GPT Transcription (or link to separate doc, if too long)",GPT Prompt,most accurate transcript?,Metadata record,Schema.org Record,CIDOC-CRM Record,prompts to GPT
22,Rumsey 3,NaN,Here is the transcription of the image titled ...,NaN,NaN,<dublin_core>\n <title>Soil Map of the Los An...,"{\n ""@context"": ""https://schema.org"",\n ""@ty...",@prefix crm: <http://www.cidoc-crm.org/cidoc-c...,NaN
23,Rumsey 4,NaN,Here is the full transcription of the map imag...,NaN,NaN,"<rdf:RDF xmlns:rdf=""http://www.w3.org/1999/02/...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","<rdf:RDF xmlns:rdf=""http://www.w3.org/1999/02/...",NaN
24,Rumsey 5,NaN,"This image shows a historical ""Post Route Map""...",NaN,NaN,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","{\n ""@context"": ""https://schema.org/"",\n ""@t...","<?xml version=""1.0"" encoding=""UTF-8""?>\n<rdf:R...",NaN
25,Rumsey 6,NaN,This image shows a historical map of Californi...,NaN,NaN,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","{\n ""@context"": ""https://schema.org/"",\n ""@t...","<?xml version=""1.0"" encoding=""UTF-8""?>\n<rdf:R...",NaN
26,Rumsey 7,NaN,"This image shows a detailed map titled ""MAP OF...",NaN,NaN,"<metadata xmlns:dc=""http://purl.org/dc/element...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","<rdf:RDF xmlns:rdf=""http://www.w3.org/1999/02/...",NaN


If there's no column with unique identifiers per row, make one:

In [32]:
col_list = list(df.columns)
identifier_col = ""
for col in col_list:
    if (col.lower() == "id") or (col.lower() == "identifier") or ("_id" in col.lower()):
        # Check that the existing IDs are, in fact, unique to each row
        existing_ids = list(df[col])
        if len(existing_ids) == len(set(existing_ids)):
            identifier_col = col
if len(identifier_col) == 0:
    # Create a unique identifier per row
    new_ids = list(range(0, df.shape[0]))
    df.insert(0, "id", new_ids)

In [33]:
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])

Rows: 27
Columns: 10


Replace the column names for clarity, consistency, and conciseness:

In [34]:
# Note: if your data header is different, replace the strings of column names before
# each colon with the column names of your data (i.e., "ID" might become "identifier").
# The strings after the colon are the new column names that the code below will expect.
if len(identifier_col) > 0:
        df.rename(columns={
        identifier_col:"id",                  # Unique identifier for each row
        "Filename":"filename", 
        "Metadata record": "dc_record",       # Dublin Core metadata record
        #"Transcription or caption (or link to separate doc, if too long)":"transcription_or_caption",
        "Schema.org Record":"sdo_record",     # Schema.org metadata record
        "CIDOC-CRM Record":"cidoccrm_record"  # CIDOC-CRM metadata record
        }, inplace=True)
else:
    df.rename(columns={
        "Filename":"filename", 
        "Metadata record": "dc_record",       # Dublin Core metadata record
        #"Transcription or caption (or link to separate doc, if too long)":"transcription_or_caption",
        "Schema.org Record":"sdo_record",     # Schema.org metadata record
        "CIDOC-CRM Record":"cidoccrm_record"  # CIDOC-CRM metadata record
        }, inplace=True)
    
df.head()


,id,filename,"Transkribus Transcription (or link to separate doc, if too long)","GPT Transcription (or link to separate doc, if too long)",GPT Prompt,most accurate transcript?,dc_record,sdo_record,cidoccrm_record,prompts to GPT
0,0,"Ann Aldrich, Mushmilling Hundred, Kent County,...",part County Deliway State\nIn quesereson taken...,Kent County Delivery State\nInquisition taken ...,"""transcribe this""",ChatGPT,<dublincore>\n <dc:title>Inquisition Report...,"{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",“create a Dublin Core metadata record”\n“creat...
1,1,"Edward Allen, No Hundred Listed, Kent County, ...",Personly Append before Jame Richarson\nCa...,Kent County\n\nPersonally appeared before Jame...,"""transcribe this""",ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": {\n ""crm"": ""http://www.cid...",“create a Dublin Core metadata record in xml”\...
2,2,"Mirandy Messick, Nanticoke Hundred, Sussex Cou...",r Mutlean Indented & taken in Nantcke\nty\nSup...,Inquest convened\nState of Alabama\n\nA pre-mi...,"""transcribe this""",ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": ""https://www.cidoc-crm.org/ci...",“create a Dublin Core metadata record in XML b...
3,3,"James Abbott, Mispillion Hundred, Kent County,...",Renl County by Anedmstion indented and take\nS...,"\nKent County, State of Delaware\n\nAn inquisi...","""transcribe this""",ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<rdf:R...","{\n ""@context"": ""http://schema.org"",\n ""@typ...","{\n ""@context"": {\n ""@vocab"": ""http://www....",“create a Dublin core record in xml based on b...
4,4,"Merritt Allee, Little Creek Hundred, Kent Coun...",Jient Crupty S.\nAn Inquiſte Aaten Le Ealeenth...,Front County S.S.\nInquisition taken at Greenw...,"""transcribe this""",ChatGPT,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<metad...","{\n ""@context"": ""https://schema.org"",\n ""@ty...","{\n ""@context"": ""https://www.cidoc-crm.org/ci...",“create a Dublin Core record in XML”\n“create ...


Create a directory to store the cleaner version of the data:

In [35]:
data_dir = data_dir + "cleaned/"
Path(data_dir).mkdir(parents=True, exist_ok=True)
df.to_csv(data_dir + f)

In [36]:
record_ids = list(df["id"])
print("Total IDs (rows):", len(record_ids))
print("Sample ID:", record_ids[2])

Total IDs (rows): 27
Sample ID: 2


### Dublin Core
Write the [Dublin Core](https://www.dublincore.org) (DC) records as XML files.

In [37]:
# Remove rows with an empty value in the dc_record column
df_dc = df[df["dc_record"].notna()]
dc_records_ids = list(df_dc["id"])

In [38]:
dc_records = list(df_dc["dc_record"])
print(dc_records[2])

<?xml version="1.0" encoding="UTF-8"?>
<metadata xmlns:dc="http://purl.org/dc/elements/1.1/">
    <dc:title>Inquest Records from Alabama, 1822</dc:title>
    <dc:creator>Jonathan Myrick (Coroner)</dc:creator>
    <dc:contributor>John Spivey, Josiah C. Nail, Thomas Howard, Harmon Howard, Mac Maxwell, William Cole, Josiah Elliott, Elias Taylor, Warner Spivey, James Roberts, Isaac Dolby, Upham Wootten, Anthony Egan, Delilah Phillips, Joseph Elliott, James Roberts</dc:contributor>
    <dc:subject>Coroner Inquests, Accidental Deaths, Historical Legal Records</dc:subject>
    <dc:description>These documents record the inquests into the deaths of Thomas Maxwell and Mirandy Myrick in Alabama in 1822. Thomas Maxwell died from injuries sustained when a sapling fell on him, while Mirandy Myrick was killed when a pine sapling, cut by her brothers, fell on her head.</dc:description>
    <dc:publisher>State of Alabama</dc:publisher>
    <dc:date>1822-04-01</dc:date>
    <dc:type>Text</dc:type>
    <

In [39]:
dc_path = data_dir+"dublin_core/"
Path(dc_path).mkdir(parents=True, exist_ok=True)

In [40]:
utils.write_xml(dc_records_ids, dc_records, dc_path, "dc_record_", ".xml")
utils.write_xml(dc_records_ids, dc_records, dc_path, "dc_record_", ".txt")

Wrote dc_record_000.xml!
Wrote dc_record_001.xml!
Wrote dc_record_002.xml!
Wrote dc_record_003.xml!
Wrote dc_record_004.xml!
Wrote dc_record_005.xml!
Wrote dc_record_006.xml!
Wrote dc_record_007.xml!
Wrote dc_record_008.xml!
Wrote dc_record_009.xml!
Wrote dc_record_015.xml!
Wrote dc_record_016.xml!
Wrote dc_record_017.xml!
Wrote dc_record_018.xml!
Wrote dc_record_019.xml!
Wrote dc_record_020.xml!
Wrote dc_record_021.xml!
Wrote dc_record_022.xml!
Wrote dc_record_023.xml!
Wrote dc_record_024.xml!
Wrote dc_record_025.xml!
Wrote dc_record_026.xml!
Wrote dc_record_000.txt!
Wrote dc_record_001.txt!
Wrote dc_record_002.txt!
Wrote dc_record_003.txt!
Wrote dc_record_004.txt!
Wrote dc_record_005.txt!
Wrote dc_record_006.txt!
Wrote dc_record_007.txt!
Wrote dc_record_008.txt!
Wrote dc_record_009.txt!
Wrote dc_record_015.txt!
Wrote dc_record_016.txt!
Wrote dc_record_017.txt!
Wrote dc_record_018.txt!
Wrote dc_record_019.txt!
Wrote dc_record_020.txt!
Wrote dc_record_021.txt!
Wrote dc_record_022.txt!


**Note:** There is inconsistency in the DC record formatting!  For example:

```
<?xml version='1.0' encoding='utf-8'?>
<dublin_core><dc element="title">Turnin' Timez: Original Student Poems</dc>
    ...
</dublin_core>
```
---
```
<?xml version="1.0"?>
<metadata xmlns:dc="http://purl.org/dc/elements/1.1/">
<dc:title>National 4-H Center Major Pledges, Contributions, and Grants</dc:title>
    ...
</metadata>
```
---
```
<?xml version="1.0" encoding="UTF-8"?>
<metadata xmlns:dcterms="http://purl.org/dc/terms/" xmlns:dc="http://purl.org/dc/elements/1.1/">
<dc:title>4-H National Youth Science Day</dc:title>
    ...
</metadata>
```
---
```
<dc:title>Climbing Up: Fun Activities for You and Your Cat</dc:title>
   ...
<dc:rights>Unknown</dc:rights>
```

Also note that [DCMI documentation](https://www.dublincore.org/specifications/dublin-core/dcmi-terms/) encourages the use of http://purl.org/dc/terms/ over http://purl.org/dc/elements/1.1/.

### Schema.org
Write the [Schema.org](https://schema.org) records as JSON-LD files.

In [60]:
# Remove rows with an empty value in the sdo_record column
df_sdo = df[df["sdo_record"].notna()]
sdo_records_ids = list(df_sdo["id"])

In [61]:
sdo_path = data_dir+"schema_org/"
Path(sdo_path).mkdir(parents=True, exist_ok=True)

In [62]:
sdo_records = list(df_sdo["sdo_record"])
print(sdo_records[2])

{
  "@context": "https://schema.org",
  "@type": "ArchiveComponent",
  "name": "Derrick A. Bell, Jr. Papers",
  "identifier": "MC.138",
  "creator": {
    "@type": "Person",
    "name": "Derrick A. Bell",
    "birthDate": "1930",
    "deathDate": "2011",
    "sameAs": "https://www.wikidata.org/wiki/Q5267493"
  },
  "description": "The collection documents the legal, academic, and civil rights work of Derrick A. Bell. It includes correspondence, writings, legal cases, teaching materials, and photographs.",
  "temporalCoverage": "1922/2011",
  "inLanguage": "en",
  "materialExtent": "189 boxes",
  "holdingArchive": {
    "@type": "ArchiveOrganization",
    "name": "New York University Archives",
    "url": "https://findingaids.library.nyu.edu/archives/mc_138/"
  },
  "accessMode": "Partially restricted under FERPA",
  "about": [
    { "@type": "Thing", "name": "Critical race theory" },
    { "@type": "Thing", "name": "Civil rights" },
    { "@type": "Place", "name": "United States" }
  ]

In [63]:
utils.write_json(sdo_records_ids, sdo_records, sdo_path, "sdo_record_", ".json")
utils.write_json(sdo_records_ids, sdo_records, sdo_path, "sdo_record_", ".txt")

Wrote sdo_record_006.json!
Wrote sdo_record_007.json!
Wrote sdo_record_008.json!
Wrote sdo_record_013.json!
Wrote sdo_record_016.json!
Wrote sdo_record_017.json!
Wrote sdo_record_018.json!
Wrote sdo_record_019.json!
Wrote sdo_record_020.json!
Wrote sdo_record_021.json!
Wrote sdo_record_022.json!
Wrote sdo_record_023.json!
Wrote sdo_record_024.json!
Wrote sdo_record_025.json!
Wrote sdo_record_006.txt!
Wrote sdo_record_007.txt!
Wrote sdo_record_008.txt!
Wrote sdo_record_013.txt!
Wrote sdo_record_016.txt!
Wrote sdo_record_017.txt!
Wrote sdo_record_018.txt!
Wrote sdo_record_019.txt!
Wrote sdo_record_020.txt!
Wrote sdo_record_021.txt!
Wrote sdo_record_022.txt!
Wrote sdo_record_023.txt!
Wrote sdo_record_024.txt!
Wrote sdo_record_025.txt!


### CIDOC-CRM
Write the [CIDOC-CRM](https://cidoc-crm.org) records as JSON-LD files.

In [64]:
# Remove rows with an empty value in the cidoccrm_record column
df_cidoc = df[df["cidoccrm_record"].notna()]
cidoc_records_ids = list(df_cidoc["id"])

In [65]:
cidoc_path = data_dir+"cidoc_crm/"
Path(cidoc_path).mkdir(parents=True, exist_ok=True)

In [66]:
cidoc_records = list(df_cidoc["cidoccrm_record"])
print(cidoc_records[2])

{
  "@context": {
    "E78": "http://www.cidoc-crm.org/cidoc-crm/E78_Collection",
    "P102_has_title": "http://www.cidoc-crm.org/cidoc-crm/P102_has_title",
    "P3_has_note": "http://www.cidoc-crm.org/cidoc-crm/P3_has_note",
    "P4_has_time-span": "http://www.cidoc-crm.org/cidoc-crm/P4_has_time-span",
    "P50_has_current_keeper": "http://www.cidoc-crm.org/cidoc-crm/P50_has_current_keeper",
    "E39": "http://www.cidoc-crm.org/cidoc-crm/E39_Actor"
  },
  "@type": "E78",
  "P102_has_title": "Derrick A. Bell, Jr. Papers",
  "P3_has_note": "Archival collection documenting Derrick Bell’s civil rights work, legal scholarship, and academic career from 1922–2011.",
  "P4_has_time-span": "1922–2011",
  "P50_has_current_keeper": {
    "@type": "E39",
    "P102_has_title": "New York University Archives"
  },
  "P14_carried_out_by": {
    "@type": "E39",
    "P102_has_title": "Derrick A. Bell"
  }
}



In [67]:
utils.write_json(cidoc_records_ids, cidoc_records, cidoc_path, "cidoccrm_record_", ".json")
utils.write_json(cidoc_records_ids, cidoc_records, cidoc_path, "cidoccrm_record_", ".txt")

Wrote cidoccrm_record_006.json!
Wrote cidoccrm_record_007.json!
Wrote cidoccrm_record_008.json!
Wrote cidoccrm_record_013.json!
Wrote cidoccrm_record_016.json!
Wrote cidoccrm_record_017.json!
Wrote cidoccrm_record_018.json!
Wrote cidoccrm_record_019.json!
Wrote cidoccrm_record_020.json!
Wrote cidoccrm_record_021.json!
Wrote cidoccrm_record_022.json!
Wrote cidoccrm_record_023.json!
Wrote cidoccrm_record_024.json!
Wrote cidoccrm_record_025.json!
Wrote cidoccrm_record_006.txt!
Wrote cidoccrm_record_007.txt!
Wrote cidoccrm_record_008.txt!
Wrote cidoccrm_record_013.txt!
Wrote cidoccrm_record_016.txt!
Wrote cidoccrm_record_017.txt!
Wrote cidoccrm_record_018.txt!
Wrote cidoccrm_record_019.txt!
Wrote cidoccrm_record_020.txt!
Wrote cidoccrm_record_021.txt!
Wrote cidoccrm_record_022.txt!
Wrote cidoccrm_record_023.txt!
Wrote cidoccrm_record_024.txt!
Wrote cidoccrm_record_025.txt!
